# PDEs 3 Workshop 5

Welcome to the final workshop of the PDE3 (Numerical) course.

This workshop has three sections:

- Section 1: Traffic Jam
- Section 2: Comparing Solvers
- Bonus section: Tsunami

In this workshop, we will solve the the 1D advection equation

$$\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0,$$

where $a$ is a constant which represents the velocity of the quantity which is being advected.

Here, we will use the advection equation to model the movement of a traffic jam; later (in the bonus section) we will use the same equation to model the progression of a tsunami.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Section 1: Traffic Jam

A 1 mile stretch of road contains a traffic jam which is $\frac{1}{2}$ mile long, starting $\frac{1}{4}$ mile from the beginning of the road. The traffic jam moves backwards along the road at a speed of 10 mph.

To simplify the model, we will assume that the traffic jam is of uniform density, and that the traffic density is 0 outside of the jam.

The instantaneous traffic density is therefore given by:

$$u(x,0) = \begin{cases} 1 &  0.25 \leq x \leq 0.75\\ 0 & \text{otherwise} \end{cases}$$

To reduce the size of the domain we need to simulate, we will use periodic boundary conditions, i.e. $u(0,t) = u(1,t)$ for all $t$.
We will implement these as part of our solver.

The ``generate_grid`` function below implements a 1D grid of $N$ uniformly spaced points in the interval $[0,1]$ which we can use to solve the advection equation.
The function returns the mesh spacing ``dx`` $(\Delta x)$ and the x values of the gridpoints ``xs``.

Run the cell to implement it.

In [ ]:
def generate_grid(N: int) -> tuple[float, np.ndarray]:
    """Generate a grid of $N$ uniformly spaced points in the interval $[0, 1]$, returning the grid spacing and the grid itself."""

    xs = np.linspace(0, 1, N, dtype=float)
    dx = xs[1] - xs[0]

    return dx, xs

### a)

Now, we need to implement the initial condition.

Write a function called ``implement_initial`` which takes the x coordinates of the grid points ``xs`` and implements the boundary conditions given above.
You should return an array ``us`` which contains the boundary conditions at each grid point.

[**Hint:** You may find the functions [``np.zeros``](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) and [``np.ones``](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) useful.]

In [ ]:
# Your code here:



Test the function to ensure it works correctly:

In [ ]:
### BEGIN TESTS ###
dx, xs = generate_grid(5)
us = implement_initial(xs)

assert us.size == 5, "The solution array has the wrong size."
assert not (np.all(np.isclose(us, np.array([0.,1.,1.,0.,0.]))) or np.all(np.isclose(us, np.array([0.,0.,1.,1.,0.])))), "The initial condition is defined using 'less than or equal to', not 'less than'."
assert np.all(np.isclose(us, np.array([0.,1.,1.,1.,0.]))), "Check to ensure that the initial conditions are implemented correctly."

# Plot the initial conditions:

_, xs_testplot = generate_grid(10000)
us_testplot = implement_initial(xs_testplot)
plt.plot(xs_testplot, us_testplot)
plt.title("Initial conditions")
plt.xlabel("$x$")
plt.ylabel("$u(x,0)$")
plt.show()

### END TESTS ###

### b)

Now that we know the initial conditions, we need to calculate an appropriate time step to use. 
This can be done using

$$
\Delta t = \nu \frac{\Delta x}{|a|}
$$

where $\nu$ is the Courant number and $a$ is the advection speed.

Write a function called ``calculate_dt`` which takes the mesh spacing ``dx`` and the advection speed ``a`` and the Courant number ``nu`` (which has a default values of ``nu=0.95``) and returns the time step ``dt``.

[**Hint:** To set default values, for example here on arg2, you can use the syntax ``def function_name(arg1, arg2=default_value)``.]

In [ ]:
# Your code here:



Test your function:

In [ ]:
assert np.isclose(calculate_dt(dx=0.1, a=1.0, nu=0.95), 0.095), "Check to ensure that the time step size is calculated correctly."
assert np.isclose(calculate_dt(dx=0.1, a=1.0), 0.095), "Check to ensure you have set the default values of the Courant number."
assert np.isclose(calculate_dt(dx=0.1, a=-1.0), 0.095), "Check to ensure that you have taken the absolute value of the advection speed."

Now we have set up the system, we are ready to solve the advection equation. To do this, we will implement the First-Order Upwind scheme, given by

$$
u_i^{n+1} = u_i^n - \frac{a\Delta t}{\Delta x} (u_{i+1}^n - u_{i}^n)
$$

where $u_i^n$ is the value of $u$ at the $i^\text{th}$ grid point at the $n^\text{th}$ timestep.

Note that we have adjusted this for backwards propogation since $a < 0$ in our case.
If we wanted to model forwards advection, we simply swap $(u_{i+1}^n - u_{i}^n)$ for $(u_{i}^n - u_{i-1}^n)$.

### c)
Write a function called ``FOU`` which implements the First-Order Upwind scheme for a single timestep.
This function takes the following arguments:

- ``us``: the current values of $u$ at each grid point
- ``a``: the advection speed
- ``dt``: the timestep
- ``dx``: the mesh spacing

The function should return the values of $u$ at the next timestep.

Remember that our domain is periodic which means that $u_{(N-1)+1}$ will be $u_{0}$.

[**Hint:** To help with the periodic boundary, you can use the [``numpy.roll``](https://numpy.org/doc/stable/reference/generated/numpy.roll.html) function, which shifts the elements of an array by a given number of positions.]

In [ ]:
# Your code here:



Test your code to ensure it works correctly:

In [ ]:
us_test = np.array([0.,1.,0.])
assert np.all(np.isclose(FOU(us_test, a=-1.0, dt=0.1, dx=1.0), np.array([0.1,0.9, 0.]))), "Check to ensure that the First-Order Upwind scheme is implemented correctly."

Now, we need to make our scheme advance in time, which we do using the ``solver`` function below.

This has been implemented for you below. Run the cell below to continue.

In [ ]:
def solver(scheme, u_0: np.ndarray, a: float, dx: float, t_total: int) -> np.ndarray:
    """Solve the advection equation using the specified scheme and return the solution array at the final time step."""
    dt = calculate_dt(dx, a)

    Nt = int(t_total // dt)

    all_us = u_0

    ts = np.arange(0, t_total, dt)

    for i in range(Nt):
        all_us = scheme(all_us, a, dt, dx)
        
    
    return all_us, ts

To get a sense of what is happening, we will start by simulating the traffic jam on a much longer stretch of road.
You don't need to understand the code in the cell below, though you can look if you would like to.

Run the cell below to continue.


In [ ]:
# Here, we take the initial conditions you have written above and embed them in a much larger domain to see how the solution evolves over time.
# You don't need to understand exactly what the code is doing here, though take a look if you would like!
# Run this cell to continue.

dx, xs = generate_grid(101)
us = implement_initial(xs)
xs_long = np.linspace(-19,1,2020, endpoint=True)
us_long = np.zeros_like(xs_long, dtype=float)
us_long[xs_long >= 0] = us

fig, ax = plt.subplots(figsize=(21, 5))

times = np.array([0,0.25,0.5,0.75,1,1.25,1.5])
colors = plt.cm.viridis(np.linspace(0, 1, times.size))
for t, color in zip(times, colors):
    bcs = us_long.copy()
    a = -10.0

    us, ts = solver(FOU, bcs, a, dx, t)
    plt.plot(xs_long, us, color=color, label=f'Time = {t}s')

norm = plt.Normalize(vmin=0, vmax=1.5)
plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='viridis'), label='Time (hours)', ax=ax)
ax.set_xlim(-19, 1)

plt.xlabel("$x$ (miles)")
plt.ylabel("Traffic Density, $u(x,t)$")
plt.title("Propagation of a traffic jam over time")
plt.show()

You should see that the traffic jam moves backwards along the road at a speed of 10 mph.
You will probably also notice that the function we used to implement the initial conditions has become smoothed over time.
We will investigate this further in the next section.

## Section 2: Comparing Schemes

Now, we can solve the advection equation on the periodic domain [0,1).

### a)
Use the functions which we have written above to investigate the behaviour of the First-Order Upwind scheme as time progresses. You should simulate the traffic jam using a grid with 5000 points and plot a comparison of the initial condition and the solution at $t=1, 2,4,$ and $8$ hours.
For each time, you should plot the initial condition and soultion on the same plot.


In [ ]:
# Your code here:



In ``schemes.py``, the Lax-Friedrichs, Lax-Wendroff and MacCormack schemes have been implemented. You do not need to understand the details of the code for this workshop, but you you can look at it if you wish.
Each of these schemes has been implemented to handle both forwards and backwards advection.

Run the cell below to import them.

If you have not been able to implement the First-Order Upwind scheme, uncomment the second line of the cell below to import it.

In [ ]:
from schemes import Lax_Friedrichs, Lax_Wendroff, MacCormack
# from schemes import FOU

### b)

Produce three sets of plots similar to the one above for the FOU scheme to explore the behaviour of the Lax-Friedrichs, Lax-Wendroff and MacCormack schemes.
You should plot the traffic density on separate plots after t = 1, 2, 4 and 8 hours. On each plot, you should also include the initial condition for comparison.

In the case of perfect advection, you should see no difference between the initial condition and the solution at later times.

In [ ]:
# Your code here:



## Section 3: Bonus Question

In this bonus question, we will model the movement of a tsunami using the advection equation.

The tsunami is modelled as a Gaussian pulse given by 

$$u(x,0) = e^{-\frac{(x-0.5)^2}{0.01}}$$

The tsunami moves at a speed of 20 km/h.

### a)

Write a function called ``implement_tsunami`` which takes in ``xs``, a numpy array of the $x$ coordinates and returns ``us `` implementing the initial condition for the tsunami.
This should work in a similar way to the ``implement_initial`` function you wrote earlier.

You should then plot the initial condition.

In [ ]:
# Your code here:



In [ ]:
dx, xs = generate_grid(1000)
us = implement_tsunami(xs)

plt.plot(xs, us)
plt.title("Initial condition")
plt.xlabel("$x$")
plt.ylabel("Tsunami height, $u(x,0)$")
plt.show()

### b)

Create a modified version of the ``FOU`` function which we wrote earlier, called ``FOU_forwards`` which implements the First-Order Upwind scheme for forwards advection.

In [ ]:
# Your code here:



### c)

Model the tsunami using the First-Order Upwind scheme for forwards advection and compare its performance with another suitable scheme from those used in Section 2.

You should use a grid with 5000 points and should model the system after 1,2,4 and 8 hours.

[**Hint:** The schemes used in Section 2 have all been written to handle both forwards and backwards advection.]

In [ ]:
# Your code here:



### d)

Discuss how the model can be made more realistic to model a tsunami wave.

<font color='orange'>Your answer here. Double click to edit.</font>

